# Comparison of MHMCMC and NUTS for Multivariate Gaussian and Ring

In [ ]:
import pymc3 as pm
import numpy as np
import matplotlib.pyplot as plt
import pickle
import os
import seaborn as sns
plt.style.use('seaborn')
sns.set_context("poster",font_scale=1)
%matplotlib inline

## 50 dimensional, highly correlated Gaussian

Creating the data for the background distribution image.

In [ ]:
N = 1000000
ndim= 50
try:
    with open("50dim0.7covar1000000.pkl", "rb") as f:
        background_y = pickle.load(f)
except:
    mv_model = pm.Model()
    cov = np.identity(ndim)
    mu = np.zeros(ndim)
    np.place(cov, cov==0, 0.7)
    with mv_model:
        y = pm.MvNormal('y', mu=mu, cov=cov, shape=(1,ndim))
        #step = pm.Metropolis()
        trace = pm.sample(N)
    #pm.traceplot(trace)
    background_y = trace.get_values("y")

In [ ]:
background_y = background_y.reshape(N, ndim)

Converting the distribution to 2 dimensional.

In [ ]:
from sklearn.decomposition import PCA
pca = PCA(n_components=2)
background_y = pca.fit_transform(background_y)

Sampling from the distribution using both methods

In [ ]:
sample_N = 500
cov = np.identity(ndim)
mu = np.zeros(ndim)
np.place(cov, cov==0, 0.7)

metr_mv_model = pm.Model()
nuts_mv_model = pm.Model()

with metr_mv_model:
    y = pm.MvNormal('y', mu=mu, cov=cov, shape=(ndim))
    step = pm.Metropolis(vars=[y])
    trace = pm.sample(sample_N/2, step=step, tune=1000)
metr_y = trace.get_values("y")
metr_y = metr_y.reshape(sample_N, ndim)
metr_y = pca.transform(metr_y)

with nuts_mv_model:
    y = pm.MvNormal('y', mu=mu, cov=cov, shape=(ndim))
    step = pm.NUTS(vars=[y])
    trace = pm.sample(sample_N/2, step=step, tune=1000)
nuts_y = trace.get_values("y")
nuts_y = nuts_y.reshape(sample_N, ndim)
nuts_y = pca.transform(nuts_y)

Plotting first 500 samples for both sampling methods.

In [ ]:
#Plotting both sampling versions
f, (ax1, ax2) = plt.subplots(2,1, sharex=True)
f.set_size_inches(12,6)
ax1.grid(False)
ax2.grid(False)
_ = ax1.hist2d(x=background_y[:,0], y=background_y[:,1], bins=100, cmap="viridis")
_ = ax1.set_xlim(-18, 18)
_ = ax1.plot(metr_y.transpose()[0,:], metr_y.transpose()[1,:],"r", lw=1.5)
_ = ax1.set_title("First 500 samples using MHMCMC")

_ = ax2.hist2d(x=background_y[:,0], y=background_y[:,1], bins=100, cmap="viridis")
_ = ax2.set_xlim(-18, 18)
_ = ax2.plot(nuts_y.transpose()[0,:], nuts_y.transpose()[1,:], "r", lw=1.5)
_ = ax2.set_title("First 500 samples using HMC-NUTS")
#f.suptitle("Sampling from a 50-d multivariate Gaussian with high covariance", fontsize=22, y=1)
f.savefig("50dim.jpg", dpi=500)


## Ring

Creating the data for the background image.

In [ ]:
ring_model = pm.Model()
N = 100000
ndim = 2

try:
    with open("ring2dim100000_v3.pkl", "rb") as f:
        background_y = pickle.load(f)
except:
    with ring_model:
        mu = pm.Normal('mu', mu=0, sd=1, shape=(ndim), testval=0.1)
        mu_circle = pm.Deterministic("mu_circle", 5*(mu/np.sqrt((mu**2).sum())))
        y = pm.MvNormal('y', mu=mu_circle, cov=np.identity(ndim)/10, shape=(1,ndim))
        step = pm.NUTS([mu, y])
        trace = pm.sample(N, step=step, tune=1000)
    background_y = trace.get_values("y")

In [ ]:
background_y = background_y.reshape(N, ndim)

Sampling from the distribution.

In [ ]:
sample_N = 250

metr_ring_model = pm.Model()
nuts_ring_model = pm.Model()

with metr_ring_model:
    mu = pm.Normal('mu', mu=0, sd=1, shape=ndim, testval=0.1)
    mu_circle = pm.Deterministic("mu_circle", 5*(mu/np.sqrt((mu**2).sum())))
    y = pm.MvNormal('y', mu=mu_circle.flatten(), cov=np.identity(ndim)/10, shape=(ndim))
    step = pm.Metropolis(vars=[mu, y, mu_circle])
    trace = pm.sample(sample_N/2, step=step, tune=1000)
metr_ring_y = trace.get_values("y")
metr_ring_y = metr_ring_y.reshape(sample_N, ndim)

with nuts_ring_model:
    mu = pm.Normal('mu', mu=0, sd=1, shape=(ndim), testval=0.1)
    mu_circle = pm.Deterministic("mu_circle", 5*(mu/np.sqrt((mu**2).sum())))
    y = pm.MvNormal('y', mu=mu_circle, cov=np.identity(ndim)/10, shape=(ndim))
    step = pm.NUTS([mu, y])
    trace = pm.sample(sample_N/2, step=step, tune=1000)
nuts_ring_y = trace.get_values("y")
nuts_ring_y = nuts_ring_y.reshape(sample_N, ndim)

Visualizing.

In [ ]:
f, (ax1, ax2) = plt.subplots(1,2, sharey=True)
f.set_size_inches(12,6)
ax1.grid(False)
ax2.grid(False)
_ = ax1.hist2d(x=background_y.transpose()[0,:], y=background_y.transpose()[1,:],bins=100, cmax=100, cmap="viridis")
_ = ax1.plot(metr_ring_y.transpose()[0,:], metr_ring_y.transpose()[1,:], "r", lw=1.5)
_ = ax1.set_title("First 500 samples using MHMCMC")

_ = ax2.hist2d(x=background_y.transpose()[0,:], y=background_y.transpose()[1,:],bins=100, cmax=100, cmap="viridis")
_ = ax2.plot(nuts_ring_y.transpose()[0,:], nuts_ring_y.transpose()[1,:], "r", lw=1.5)
_ = ax2.set_title("First 500 samples using HMC-NUTS")
#f.suptitle("Sampling from a 2-d annulus", fontsize=22, y=1)
f.savefig("ring.jpg", dpi=500)